## Final Project Submission

Please fill out:
* Student name: Wesley Yu
* Student pace: Flex
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


enter image here?

![example](images/director_shot.jpeg)

# King County Home Regression Analysis

**Authors:** Wesley Yu
***

## Overview

This project will use multiple linear regression to analyze house sales in King County WA.

## Business Problem

A real estate company is looking to expand into the King County market. They want to know what kind of features will affect house prices in this area so that they can advise potential home buyers.

## Data Understanding

Data set from King County contains 21,597 records of house sales during 2014 - 2015 period. Data also contains sale prices, various features describing the homes, condition of property when sold, and location details. Additional data from US Post office was added to help narrow down zipcodes.


## Data Preparation

### EDA 



In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
import folium
import branca.colormap as cm

In [ ]:
df = pd.read_csv('data/kc_house_data.csv')
df.head()

In [ ]:
df.info()

In [ ]:
# check for duplicates
df.duplicated(subset = 'id').sum()

In [ ]:
df[df.duplicated(subset = 'id', keep = False)]

In [ ]:
df.drop_duplicates(subset = 'id', keep = 'last', inplace = True)

Duplicates were found to be homes that were sold twice during 2014-2015 period. I decided to keep most recent records of house sales to reflect the changing market.

In [ ]:
# check for nulls
df.isna().sum()

In [ ]:
df.waterfront.value_counts()

In [ ]:
df.waterfront.fillna('NO', inplace = True)

In [ ]:
df.waterfront.replace({'NO': 0, 'YES':1}, inplace = True)

I decided to replace null values in waterfall to 'NO' and convert binary values for inputting into model.

In [ ]:
df.view.value_counts()

In [ ]:
df.view.fillna('NONE', inplace = True)

In [ ]:
df.view.replace({'NONE': 0, 'FAIR': 1, 'AVERAGE': 2, 'GOOD':3, 'EXCELLENT': 4}, inplace = True)

Same was done to view, and replaced with ordinal values with numeric

In [ ]:
df.yr_renovated.value_counts()

In [ ]:
df.drop('yr_renovated', axis = 1, inplace = True)

There is a large number of nulls in the yr_renovated feature. Majority of values are also 0. We will drop this column.

In [ ]:
df.sqft_basement.value_counts()

In [ ]:
df.sqft_basement.replace('?', 0, inplace = True)
df.sqft_basement = df.sqft_basement.astype(float)

In [ ]:
(df.sqft_basement + df.sqft_above == df.sqft_living).sum()

In [ ]:
df.loc[(df.sqft_basement + df.sqft_above != df.sqft_living), 'sqft_basement'] = 1

In [ ]:
df.loc[df.sqft_basement > 0, 'sqft_basement'] = 1

In [ ]:
df.rename(columns = {'sqft_basement': 'basement'}, inplace = True)

Replacing '?' with 0. It looks like sqft_basement + sqft_above = sqft_living for almost every value. Changed sqft_basement to new column with binary values, 0 for no basement and 1 for basement. The 170 records where sqft_basement + sqft_above != sqft_living will be assign to 1 (has basement).

In [ ]:
df.condition.value_counts()

In [ ]:
df.condition.replace({'Poor': 1, 'Fair': 2, 'Average': 3, 'Good':4, 'Very Good': 5}, inplace = True)

Condition values changed from ordinal to numeric for modeling purposes.

In [ ]:
df.grade.value_counts()

In [ ]:
df.grade = df.grade.str.extract('(\d+)').astype(int)

Grade feature we will keep only digit values and convert to numeric.

In [ ]:
df.describe()

Quick summary shows an odd outlier for bedrooms. Upon examination this could be a typo, I changed value to 3.

In [ ]:
df[df.bedrooms == 33]

In [ ]:
df.loc[df.bedrooms == 33, 'bedrooms'] = 3

In [ ]:
df.bathrooms.value_counts()

In [ ]:
df.bathrooms = df.bathrooms.map(np.ceil)

In [ ]:
df.bathrooms.unique()

I decide to treat 1/4, 1/2, and 3/4 bathrooms as whole bathrooms, for easier interpretation.

### Dealing with outliers

In [ ]:
# take a quick look at distribution of each variable
df.hist(figsize = (20,20));

For model optimization, i decide to remove any values in feature that represent less than 1% of the data.

In [ ]:
df.bedrooms.value_counts(normalize = True)

In [ ]:
df.drop(df[(df.bedrooms < 2) | (df.bedrooms > 6)].index, inplace = True)

In [ ]:
df.bathrooms.value_counts(normalize = True)

In [ ]:
df.drop(df[df.bathrooms > 4].index, inplace = True)

In [ ]:
df.condition.value_counts(normalize = True)

In [ ]:
df.drop(df[df.condition < 3].index, inplace = True)

In [ ]:
df.grade.value_counts(normalize = True)

In [ ]:
df.drop(df[(df.grade < 5) | (df.grade > 11)].index, inplace = True)

In [ ]:
df.info()

In [ ]:
(df.sqft_lot > 200000).sum()

In [ ]:
df.drop(df[df.sqft_lot > 200000].index, inplace = True)

In [ ]:
counts = df.zipcode.value_counts(normalize = True)

In [ ]:
df = df[~df.zipcode.isin(counts[counts < 0.01].index)]

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.hist(figsize = (20,20));

Creating a map with all records of houses marked, to get an idea if price is affected by location.

In [ ]:
# getting coordinates for the center of map
centerlat = (df['lat'].max() + df['lat'].min()) / 2
centerlong = (df['long'].max() + df['long'].min()) / 2
center = (centerlat, centerlong)

In [ ]:
# create a LinearColorMap and assign colors, vmin, and vmax
# the colormap will show green for $100,000 homes all the way up to red for $1,500,000 homes
colormap = cm.LinearColormap(colors=['green', 'yellow', 'red'], vmin=100000, vmax=1500000)

# create our map again.  This time I am using a different tileset for a new look
m = folium.Map(location=center, zoom_start=10, tiles = 'Stamen Toner')

# Same as before... go through each home in set, make circle, and add to map.
# This time we add a color using price and the colormap object
for i in range(len(df)):
    folium.Circle(
        location=[df.iloc[i]['lat'], df.iloc[i]['long']],
        radius=10,
        fill=True,
        color=colormap(df.iloc[i]['price']),
        fill_opacity=0.2
    ).add_to(m)

# the following line adds the scale directly to our map
m.add_child(colormap)

We can see location has an effect of the price. Further south of Seattle shows the largest area of cheaper houses.

### Feature Engineering


Create a new column representing age of the property.

In [ ]:
df.date = pd.to_datetime(df.date)
df['house_age'] = df.date.dt.year - df.yr_built

To help cut down the size of unique zipcode variables, I bring in data on which cities the zipcodes belong to.

In [ ]:
df.zipcode.value_counts().size

In [ ]:
data = pd.read_csv('data/zip_code_database.csv')
zip_king = data[(data.county == 'King County') & (data.state == 'WA') & (data['type'] == 'STANDARD')]

In [ ]:
zip_king.head()

In [ ]:
king_cities = zip_king[['zip', 'primary_city']]

In [ ]:
df = pd.merge(df, king_cities, left_on='zipcode', right_on='zip', how='left') 

In [ ]:
df.head()

Removing unused columns

In [ ]:
todrop = ['id', 'date', 'lat', 'long', 'zipcode', 'zip', 'yr_built']

In [ ]:
df.drop(todrop, axis = 1, inplace = True)

### Feature Selection

Linear regression is a great method to determine the strength of predictors. In order for linear regression to produce trustworthy results there are certain assumptions that need to be followed: 
* Linearity - relationship between predictor and target should be linear.
* Independence - Observations are independent from each other, low or no multicollinearity.
* Normality - Errors should be normally distributed
* Homoscedasticity - variance or the errors is the same

First we check the assumption of linearity by inspecting the scatter plots of the predictors with the target.

In [ ]:
x = df.drop('price', axis = 1)
y = df.price

fig, axes = plt.subplots(ncols=2, nrows=6, figsize=(10, 20))
fig.set_tight_layout(True)
fig.delaxes(axes[5,1])

for index, col in enumerate(x.columns):
    ax = axes[index//2][index%2]
    ax.scatter(x[col], y, alpha=0.2)
    ax.set_xlabel(col)
    ax.set_ylabel("price")

A linear relationship is not visible for house_age, bedrooms, floors, sqft_lot.

Checking with boxplots for clarity shows linear relationship with price.

In [ ]:
sns.boxplot(x = 'bedrooms', y = 'price', data = df);

In [ ]:
sns.boxplot(x = 'view', y = 'price', data = df);

In [ ]:
sns.boxplot(x = 'floors', y = 'price', data = df);

Log transformation of price and sqft_living improves linearity, will apply this during modeling.

In [ ]:
x = np.log(df.sqft_living)
y = np.log(df.price)
sns.scatterplot(x=x,y=y);

Next we will check for multicollinearity between our predictors. This can be done by inspecting correlation of each predictor. Using a heatmap of the correlation matrix is a helpful visual.

In [ ]:
corr = df.drop('price', axis = 1).corr()

In [ ]:
# creating heatmap of correlation matrix
fig, ax = plt.subplots(figsize=(20, 20))
sns.heatmap(data=corr, mask=np.triu(np.ones_like(corr, dtype=bool)), ax=ax,\
            annot=True,);

Generally 0.7 - 0.8 or higher absolute value of correlation is considered high. We will take anything above 0.75 as a cut-off.

In [ ]:
# Construct pairs based on their correlation values
corr_list = corr.abs().stack().reset_index().sort_values(0, ascending=False)
corr_list['pairs'] = list(zip(corr_list.level_0, corr_list.level_1))
corr_list.set_index(['pairs'], inplace = True)
corr_list.drop(columns=['level_1', 'level_0'], inplace = True)
corr_list.columns = ['cc']
corr_list.drop_duplicates(inplace=True)
corr_list[(corr_list.cc > .75) & (corr_list.cc < 1)]

sqft_above, sqft_living, sqft_lot15, sqft_lot, and sqft_living15 are found to be highly correlated variables. 
Removing all but sqft_living and sqft_lot should help with multicollinearity problems.

In [ ]:
df.drop(['sqft_above', 'sqft_lot15', 'sqft_living15'], axis = 1, inplace = True)

To start our baseline model we will select variables that have the highest correlation with our target price, which is found to be sqft_living and grade.

In [ ]:
df.corr().price.abs().sort_values(ascending = False)

Splitting data set to train/test data sets for model validation later on.

In [ ]:
X = df.drop('price', axis = 1)
y = df.price

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [ ]:
print(X_train.shape, y_train.shape)

Split categorical variables from numeric

In [ ]:
X_train_num = X_train.drop('primary_city', axis = 1)

In [ ]:
X_train_num.head()

In [ ]:
X_train_cat = X_train.primary_city

Using pandas to create dummy variables for primary_city. Dropping a column to avoid multicolinearity.

In [ ]:
dummies = pd.get_dummies(X_train_cat)
dummies.drop('Seattle', axis = 1, inplace = True)
X_train_cat = pd.concat([X_train_cat, dummies], axis = 1)
X_train_cat.drop('primary_city', axis = 1, inplace = True)
X_train_cat.head()

## Modeling

### Baseline Model

Baseline model will consist of the 2 highest correlated variables to our target

In [ ]:
high_corr = ['sqft_living', 'grade']
firstX = X_train_num[high_corr]

In [ ]:
model = sm.OLS(y_train, sm.add_constant(firstX)).fit()
model.summary()

R^2 value of 0.488 shows that only around 49% of prices variance is explained by sqft living and grade. 
p-values for all predictors round to 0, which indicates that they have statistical significance.
coef for sqft_living shows for each unit of increased in sqft_living, price will increase by 151.25
same for grade with an increase of 98,130 in price for each unit of grade increase.

#### Check for normality of residuals with QQ plot.

In [ ]:
preds = model.predict(sm.add_constant(firstX))
residuals = model.resid
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True);

Check for homoscedasticity in residuals with scatterplot.

In [ ]:
y_pred = model.predict(sm.add_constant(firstX))

In [ ]:
sns.residplot(x = y_pred, y = residuals);

Assumptions for normality and homoscedasticity are found to be false. This can be corrected by log transformation of the target 'price', as its distribution was right skewed.

In [ ]:
y_train_log = np.log(y_train)
firstX_log = pd.DataFrame({})
firstX_log['grade'] = X_train_num.grade
firstX_log['sqft_living_log'] = np.log(X_train_num.sqft_living)

In [ ]:
model_log = sm.OLS(y_train_log, sm.add_constant(firstX_log)).fit()
model_log.summary()

After log transformation of our target coef for predictors are now in terms of log.
Each percent of unit increase of sqft_living will increase price by 0.42%
unit increase in grade will result in a 19% increase in price.
R^2 value has also increased a bit after the transformation as well.

In [ ]:
preds = model_log.predict(sm.add_constant(firstX_log))
residuals = model_log.resid
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True);

In [ ]:
y_pred = model_log.predict(sm.add_constant(firstX_log))
sns.residplot(x = y_pred, y = residuals);

Assumptions for normality and homoscedasticity are now satisfied.

### Model 2

Adding moderately correlated variables to see if we can increase R^2 score.

In [ ]:
corr_data = pd.concat([y_train, X_train_num], axis=1)
corr_data.corr().price.abs().sort_values(ascending = False)

In [ ]:
moderate = ['bathrooms', 'view', 'waterfront', 'bedrooms']
secondX = pd.concat([firstX_log, X_train_num[moderate]], axis=1)

In [ ]:
model2 = sm.OLS(y_train_log, sm.add_constant(secondX)).fit()
model2.summary()

R^2 has increased a bit to 0.534
p-values round to 0
assumptions for normality and homoscedasticity still hold.
Interestingly additional units of bedrooms and bathrooms decrease the price by around 4% for bathrooms and 2% bedrooms if sqft_living remains the same. We will add on our variables to account for location in next model to see if this changes.

In [ ]:
preds = model2.predict(sm.add_constant(secondX))
residuals = model2.resid
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True);

In [ ]:
y_pred = model2.predict(sm.add_constant(secondX))
sns.residplot(x = y_pred, y = residuals);

### Model 3

Adding in our dummy encoded variables to see if location can account for any variance in the data.

In [ ]:
thirdX = pd.concat([secondX, X_train_cat], axis=1)

In [ ]:
model3 = sm.OLS(y_train_log, sm.add_constant(thirdX)).fit()
model3.summary()

R^2 score has increased to 0.737
p=value for bathrooms now 0.355 indicating that this predictor may not be significant, we can consider dropping for next model.
p-value for federal way also showing to be barely non significant at 0.057.
Since we dropped Seattle dummy variable. coeffiencents for listed cities are in reference to Seattle. 
the price of a similar house in Auburn is around 58% cheaper than seattle.

In [ ]:
preds = model3.predict(sm.add_constant(thirdX))
residuals = model3.resid
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True);

In [ ]:
y_pred = model3.predict(sm.add_constant(thirdX))
sns.residplot(x = y_pred, y = residuals);

### Model 4

Instead of manually going through models of different variables. We can take advantage of RFE, recurrsive feature elimination. 

In [ ]:
X_train_num.sqft_living = np.log(X_train_num.sqft_living)

In [ ]:
rfeX = pd.concat([X_train_num, X_train_cat], axis = 1)

In [ ]:
lr_rfe = LinearRegression()
select = RFE(lr_rfe, n_features_to_select=20)

In [ ]:
select.fit(X=rfeX, y=y_train_log)

In [ ]:
features = select.support_

In [ ]:
finalX = rfeX[rfeX.columns[features]]

In [ ]:
model4 = sm.OLS(y_train_log, sm.add_constant(finalX)).fit()
model4.summary()

strong R^2 of 0.747, adjusted R^2 also high. 
p-values of all predictors round to 0
Strongest predictors show to be location based. followed by grade and condition of the house.

In [ ]:
np.sqrt(model4.mse_resid)

In [ ]:
model4.params

In [ ]:
preds = model4.predict(sm.add_constant(finalX))
residuals = model4.resid
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True);

In [ ]:
y_pred = model4.predict(sm.add_constant(finalX))
sns.residplot(x = y_pred, y = residuals);

### Model 5

In [ ]:
all_var = pd.concat([X_train_num, X_train_cat], axis = 1)

In [ ]:
model5 = sm.OLS(y_train_log, sm.add_constant(all_var)).fit()
model5.summary()

Although R^2 score is higher in this model, condition number is very high as well. this is an indicator that there may be strong multicollinearity between the predictors and may affect the coefficient values. We will go with model 4 as our final model.

## Model validation

Final step is to test our model with new data and see how well it performs

In [ ]:
y_test_log = np.log(y_test)
X_test.sqft_living = X_test.sqft_living.map(np.log)

In [ ]:
X_test_num = X_test.drop('primary_city', axis = 1)
X_test_cat = X_test.primary_city

dummies = pd.get_dummies(X_test_cat)
dummies.drop('Seattle', axis = 1, inplace = True)

X_test_final = pd.concat([X_test_num, dummies], axis = 1)
X_test_final.head()

In [ ]:
finaltestX = X_test_final[X_test_final.columns[features]]

In [ ]:
y_pred_train = model4.predict(sm.add_constant(finalX))
y_pred_test = model4.predict(sm.add_constant(finaltestX))

In [ ]:
train_mse = mean_squared_error(y_train_log, y_pred_train)
test_mse = mean_squared_error(y_test_log, y_pred_test)
print('Train Mean Squarred Error:', train_mse)
print('Test Mean Squarred Error:', test_mse)

MSE for train and test do not show a big difference, which indicates our model performed similarly on the new data 

## Conclusions



- __Location, location, location__ Our final model showed that the biggest factor in house prices are location. results from analysis show as you move futher south(away) from Seattle, prices for a house with similar features decrease.

- __House Quality__ Aside from location, the next biggest factor in house prices are the condition and grade of the property.

- __Limitations__ 

 
## Next Steps

- __House Features__ We can group data by city only and see if specific features of house ie. Number of bathrooms or bedrooms have an affect on house price.

